In [6]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
data = dataiku.Dataset("new_train_data")
df = data.get_dataframe()

In [7]:
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import time

import os

In [9]:
df

,date,broker-102/data,broker-104/data,broker-106/data,broker-108/data,broker-110/data,broker-101/data,broker-103/data,broker-105/data,broker-107/data,broker-109/data
0,2021-08-25 18:00:00+00:00,88.057892,77.800600,72.025539,75.970707,95.002221,83.629912,78.380594,73.159848,80.008438,94.650912
1,2021-08-25 17:45:00+00:00,88.147967,77.907399,72.060813,76.024023,95.030465,83.564348,78.348473,73.171065,80.077924,94.749819
2,2021-08-25 17:30:00+00:00,88.385463,78.065120,72.024769,76.106181,95.000380,83.634059,78.468809,73.136793,80.014919,94.751367
3,2021-08-25 17:15:00+00:00,88.264818,78.101756,72.106583,76.138057,95.005182,83.613774,78.464204,73.180682,80.090870,94.790193
4,2021-08-25 17:00:00+00:00,88.373786,77.969001,72.127202,76.129132,95.132078,83.712252,78.579221,73.189930,80.152515,94.886214
5,2021-08-25 16:45:00+00:00,88.332493,78.094722,72.160736,76.170051,95.228092,83.685842,78.725486,73.296315,80.231704,94.853790
6,2021-08-25 16:30:00+00:00,88.378953,78.163383,72.232277,76.226302,95.313764,83.751507,78.825091,73.411269,80.181206,94.943533
7,2021-08-25 16:15:00+00:00,88.478915,78.215185,72.397890,76.351413,95.344451,83.725933,78.882662,73.522625,80.367834,94.975855
8,2021-08-25 16:00:00+00:00,88.446663,78.198984,72.367600,76.496676,95.429872,83.850682,78.833776,73.452424,80.440068,95.023090
9,2021-08-25 15:45:00+00:00,88.403636,78.354263,72.514395,76.451271,95.398651,83.753287,78.934004,73.408386,80.497241,94.998835


# Split en dataset de Train et Test

In [8]:
# Split into training, test datasets.
# Since it's timeseries we should do it by date.
test_cutoff_date = df_data['date'].max() - timedelta(days=7)

test_df = df[df['date'] > test_cutoff_date]
train_df = df[df['date'] <= test_cutoff_date]

#check out the datasets
print('Test dates: {} to {}'.format(test_df['date'].min(), test_df['date'].max()))
print('Train dates: {} to {}'.format(train_df['date'].min(), train_df['date'].max()))

Test dates: 2021-08-25 15:45:00+00:00 to 2021-08-25 20:30:00+00:00
Train dates: NaT to NaT


In [0]:
train_df  = train_df.set_index('date')
test_df = test_df.set_index('date')

In [0]:
train = train_df
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df
for i in test_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

# Construction des mini-batch

In [0]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [0]:
n_past = 10
n_future = 5
n_features = 10

In [0]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

# Création du modèle de forecasting

In [0]:
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs = encoder(encoder_inputs)

encoder_states = encoder_outputs[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs[0])

#
decoder = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states)
decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder)

#
model = tf.keras.models.Model(encoder_inputs,decoder_outputs)

#
model.summary()

# Apprentissage du modèle

In [0]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber(), metrics=[tf.keras.metrics.CosineSimilarity(), tf.keras.metrics.MeanAbsoluteError()])
history = model.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])

In [0]:
print(history.history.keys())
history.history["val_mean_absolute_error"]

data_metrics = {}

for key, val in history.history.items():
    data_metrics[key] = [val[-1]]

print(data_metrics)

In [0]:
for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

# Stockage du modèle dans le folder

In [0]:
model_json = model.to_json()

In [0]:
# Write recipe outputs
model_folder = dataiku.Folder("dHoUQGRB")
model_folder_info = model_folder.get_info()

now = time.time()

model_folder.write_json(str(now)+"/model_json", model_json)
model_folder.write_json("actual/model_json", model_json)

In [0]:
model_folder.list_paths_in_partition()

# Stockage des métriques dans un dataframe

In [0]:
metrics = dataiku.Dataset("Metrics")

data_metrics['time'] = [now]
data_metrics["used"] = ["True"]
df_metrics = pd.DataFrame(data_metrics)

metrics.write_with_schema(df_metrics)